# Twitter extraction

This notebook aims to retrieve tweets, clean them and compute a sentiment in order to observe a correlation between crypto currencies and tweets' sentiments. The following steps are executed in this notebook :

- Retrieve tweets with Twython API (Twitter API wrapper for python)
- Extract the wanted data (tweet's text, #followers, #likes, etc.)
- Clean the textual data (remove unnecessary elements like media, websites link, pseudos, ...)
- Compute for each tweet a sentiment score with Vader (named compound) and a score linked to the popularity of the tweet and its compound

This notebook is written using Python 3.6.

## Setup

In [43]:
# Define the currency
#CURRENCY = "zilliqa"
#CURRENCY_SYMBOL = "ZIL"
#CURRENCY = "nexo"
#CURRENCY_SYMBOL = "NEXO"
CURRENCY = "bitcoin"
CURRENCY_SYMBOL = "BTC"

## personal config
TWEETS_FOLDER    = f"data/crypto/{CURRENCY_SYMBOL}" # Relative path to historical data
SEP_CHAR         = '~' # character seperating dates from and to in filename
ENVS             = ['CRYPTO', 'LINE_COUNT', 'MOST_RECENT_FILE', 'MOST_RECENT_ID'] # Stored in var.csv
MAX_ROW_PER_FILE = 20000 # Each file storing data has a maximum amount of rows

tweets_raw_file = f'data/twitter/{CURRENCY_SYMBOL}/{CURRENCY}_tweets_raw.csv'
tweets_clean_file = f'data/twitter/{CURRENCY_SYMBOL}/{CURRENCY}_tweets_clean.csv'
query = f'#{CURRENCY} OR #{CURRENCY_SYMBOL}' ####TODO PUT BACK  OR {CURRENCY} OR ${CURRENCY} OR ${CURRENCY_SYMBOL}

## 1. Retrieve the tweets from Twitter API

### 1.1 Import Twython
We use the *twython* package as my Python interface with the Twitter API: https://twython.readthedocs.io/en/latest/usage/starting_out.html

The twython package must be installed using *pip install twython* from the command line.

In [44]:
from twython import Twython

### 1.2 OAuth2 Authentication (*app* authentication)
Here we use the method *OAuth2* along with the Twithon library to authenticate on the twitter API.

OAuth1 will give you *user* access to the API, whereas OAuth2 will give the *app* access. For academic use the rate limits are generally better for *OAuth2* (app) authentication, with a few exceptions. For a chart showing the API limits for user and app authentication for the various parts of the Twitter API, see this chart: https://dev.twitter.com/rest/public/rate-limits

Running the code block below shows that we now have a rate limit of 450 API calls. This means we can make 450 different calls to the API within the current 15-minute window. With the search API we can access 100 tweets per call. This means that, if we were downloading tweets with a specific hashtag, such as *#arnova16*, we could download 450 $\times$ 100 or 45,000 tweets per window. This is much better than the 18,000 tweets we can access using the OAuth1 or user authentication.

In [45]:
APP_KEY = 'saGd6QPYxZ7ZIAY47uQNOga7H'
APP_SECRET = '18wzJnroHgtR3NRSHuumm0o2xffC5dneR6SXdoleuifYt72OwU'
twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)
twitter.get_application_rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 450, 'remaining': 440, 'reset': 1613069827}}

### 1.3 Query the twitter API
Here we query the twitter API to get the latest tweets about bitcoin. Then we transform it to store only the useful data inside a Pandas Dataframe.

The following fields are retrieved from the response:

- **id** (int) : unique identifier of the tweet
- **text** (string) : UTF-8 textual content of the tweet, max 140 chars
- user
  - **name** (string) : twitter's pseudo of the user
  - **followers_count** (int) : Number of followers the user has
- **retweet_count** (int) : Number of times the tweet has been retweeted
- **favorite_count** (int) : Number of likes
- **created_at** (datetime) : creation date and time of the tweet

Also, we wanted to retrieve the following fields but it is not possible with the standard free API, Enteprise or premium is needed (https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html):

- reply_count (int) : Number of times the Tweet has been replied to

The pandas package must be installed using *pip install pandas* from the command line.

We used the search opertators that are explained here (https://lifehacker.com/search-twitter-more-efficiently-with-these-search-opera-1598165519) to not only search by hashtag but also the tweets that contain the currency name or that have the hashtag with the currency's abreviation.

In [46]:
from time import sleep
import json
import pandas as pd
import io
from tqdm import tqdm

In [47]:
NUMBER_OF_QUERIES = 10
data = {"statuses": []}
next_id = None
with open(tweets_raw_file,"a+", encoding='utf-8') as f:
    if not next_id:
        f.write("ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt\n")
    while(True):
        twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)
        last_size = 0
        for i in tqdm(range(NUMBER_OF_QUERIES)):
            if not next_id:
                data = twitter.search(q=query, lang='en', result_type='recent', count="100") # Use since_id for tweets after id
            else:
                data["statuses"].extend(twitter.search(q=query, lang='en', result_type='mixed', count="100", max_id=next_id)["statuses"])
            if len(data["statuses"]) > 1:
                next_id = data["statuses"][len(data["statuses"]) - 1]['id']
            if last_size + 1 == len(data["statuses"]):
                break
            else:
                last_size = len(data["statuses"])

        print('Retrieved {0}, waiting for 15 minutes until next queries'.format(len(data["statuses"])))
        #d = pd.DataFrame([[s["id"], s["text"].replace('\n','').replace('\r',''), s["user"]["name"], s["user"]["followers_count"], s["retweet_count"], s["favorite_count"], s["created_at"]] for s in data["statuses"]], columns=('ID', 'Text', 'UserName', "UserFollowerCount", 'RetweetCount', 'Likes', "CreatedAt"))
        #d.to_csv(f, mode='a', encoding='utf-8',index=False,header=False)
        #if last_size + 1 == len(data["statuses"]):
         #   print('No more new tweets, stopping...')
          #  break
        data["statuses"] = []
        break
        sleep(910)

100%|██████████| 10/10 [00:07<00:00,  1.42it/s]

Retrieved 996, waiting for 15 minutes until next queries


## Preprocessing

Now we will cleanup the data.

We already filtered tweets in english in the call to the Twitter API.
We will now filter links, @Pseudo, images, videos, unhashtag #happy -> happy.

We won't transform to lower case because Vader take capital letters into consideration to emphasize sentiments.

You must install `pip install tqdm`

In [48]:
import re # regular expressions
from tqdm import tnrange, tqdm_notebook, tqdm

d = pd.read_csv(tweets_raw_file)
for i,s in enumerate(tqdm(d['Text'])):
    text = d.loc[i, 'Text']
    text = text.replace("#", "")
    text = re.sub('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', text, flags=re.MULTILINE)
    text = re.sub('@\\w+ *', '', text, flags=re.MULTILINE)
    d.loc[i, 'Text'] = text
f = open(tweets_clean_file, 'a+', encoding='utf-8')
d.to_csv(f, header=True, encoding='utf-8',index=False)

100%|██████████| 1001/1001 [00:00<00:00, 3115.13it/s]


In [49]:
df_clean = pd.read_csv(tweets_clean_file)
df_clean.head(5)

,ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt
0,1359455535255285769,RT : Don't trust banks. Always buy Bitcoin.,Raul,74,2,0,Wed Feb 10 10:53:51 +0000 2021
1,1359455532055199744,$aave is not done yet...the supply is so low w...,Cryptopassion,35,0,0,Wed Feb 10 10:53:50 +0000 2021
2,1359455528238387200,RT : 🔥TOTAL CRYPTO MARKETCAP BREAK OUT🔥🔷 This ...,Chad,14,105,0,Wed Feb 10 10:53:50 +0000 2021
3,1359455523704217601,My ETH 0x65862B695E11058884058a17C33bFCCFc6b1b...,Soni,32,0,0,Wed Feb 10 10:53:48 +0000 2021
4,1359455523385524224,RT : SNAKE Token Airdrop Live!!! Airdrop 2nd ...,Malibu_Mo,9,4716,0,Wed Feb 10 10:53:48 +0000 2021


In [50]:
df_clean.min(axis=0)

ID                                                 1359452740607332354
Text                 "One year's worth of bitcoin electricity consu...
UserName                                                #100DaysOfCode
UserFollowerCount                                                    0
RetweetCount                                                         0
Likes                                                                0
CreatedAt                                                    CreatedAt
dtype: object

In [51]:
df_clean.max(axis=0)

ID                                                                  ID
Text                 🤝 Follow me on . Let's hunt for Bitcoins toget...
UserName                                                           🦋🦋🦋
UserFollowerCount                                    UserFollowerCount
RetweetCount                                              RetweetCount
Likes                                                            Likes
CreatedAt                               Wed Feb 10 10:53:51 +0000 2021
dtype: object

In [52]:
df_clean = df_clean[df_clean['ID']!='ID']

In [53]:
df_clean['Text'][973]

'Tesla Invests $1.5 Billion in Bitcoin: Price Rallies - BTC ElonMusk - /HYeD7to8z3 /kKUG7Ku7vO'

In [54]:
df_clean.sort_values('ID')

,ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt
2998,1359452740607332354,RT : We continued to work with global industry...,jejes🐰🐰,83,942,0,Wed Feb 10 10:42:45 +0000 2021
1997,1359452740607332354,RT : We continued to work with global industry...,jejes🐰🐰,83,942,0,Wed Feb 10 10:42:45 +0000 2021
997,1359452740607332354,RT : We continued to work with global industry...,jejes🐰🐰,83,942,0,Wed Feb 10 10:42:45 +0000 2021
2997,1359452756755402754,Ideaology has listed on Uniswap! $IDEA /wcEisv...,Rakib Shakot,546,0,0,Wed Feb 10 10:42:49 +0000 2021
1996,1359452756755402754,Ideaology has listed on Uniswap! $IDEA /wcEisv...,Rakib Shakot,546,0,0,Wed Feb 10 10:42:49 +0000 2021
...,...,...,...,...,...,...,...
1001,1359455532055199744,$aave is not done yet...the supply is so low w...,Cryptopassion,35,0,0,Wed Feb 10 10:53:50 +0000 2021
1000,1359455535255285769,RT : Don't trust banks. Always buy Bitcoin.,Raul,74,2,0,Wed Feb 10 10:53:51 +0000 2021
2001,1359455535255285769,RT : Don't trust banks. Always buy Bitcoin.,Raul,74,2,0,Wed Feb 10 10:53:51 +0000 2021
3002,1359455535255285769,RT : Don't trust banks. Always buy Bitcoin.,Raul,74,2,0,Wed Feb 10 10:53:51 +0000 2021


In [55]:
f.close()